In [3]:
import requests
import json
from tqdm import tqdm
import chess.pgn
import io
from stockfish import Stockfish

stockfish_path = r"C:\Users\remik\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"
stockfish = Stockfish(path=stockfish_path)


def get_chess_com_games(username, game_type='rapid'):
    response = requests.get(f"https://api.chess.com/pub/player/{username}/games/archives")
    archives = response.json()['archives']

    games = []
    for url in tqdm(archives):
        response = requests.get(url)
        game_infos = response.json()['games']
        for game_info in game_infos:
            if game_info['time_class'] == game_type:
                games.append(game_info)
    return games

def get_my_color(player_data, my_username):
    if player_data['white']['username'] == my_username:
        return True  # True means white
    elif player_data['black']['username'] == my_username:
        return False  # False means black
    else:
        return None

def get_top_moves(fen, depth=10, top_moves=3):
    stockfish.set_fen_position(fen)
    stockfish.set_depth(depth)
    top_moves_data = stockfish.get_top_moves(top_moves)
    top_moves_scores = {move['Move']: move['Centipawn'] for move in top_moves_data}
    return top_moves_scores

def get_best_move(fen, depth=10):
    stockfish.set_fen_position(fen)
    stockfish.set_depth(depth)
    best_move = stockfish.get_best_move()
    best_move_score = stockfish.get_evaluation()['value']
    return {best_move: best_move_score}

def analyze_game(game_data, username="fanneett00669", filename="exercises.json"):
    pgn_text = game_data['pgn']
    game = chess.pgn.read_game(io.StringIO(pgn_text))
    board = game.board()
    my_color = get_my_color(game_data, username)
    url = game_data['url']
    my_elo = game_data['white']['rating'] if my_color else game_data['black']['rating']
    opponent_elo = game_data['black']['rating'] if my_color else game_data['white']['rating']

    # Load existing exercises
    try:
        with open(filename, 'r') as f:
            exercises = json.load(f)
    except FileNotFoundError:
        exercises = []

    for i, move in enumerate(game.mainline_moves()):
        if i >= 16:
            break

        fen = board.fen()
        top_moves = get_top_moves(fen, depth=15, top_moves=3)
        my_move = str(move)

        # Make the "bad" move
        board.push(move)
        # Get best opponent move after the "bad" move
        opponent_best_move = get_best_move(board.fen(), depth=10)
        opponent_best_move_score = list(opponent_best_move.values())[0]  # Get the score
        #print(top_moves, opponent_best_move_score)
        if board.turn != my_color and abs(opponent_best_move_score - max(top_moves.values())) > 50:
            exercise = {
                'position': fen,
                'wrong_move': my_move,
                'top_moves': top_moves,
                'url': url,
                'my_color': 'white' if my_color else 'black',
                'my_elo': my_elo,
                'opponent_elo': opponent_elo,
                'move_number': i,
                'opponent_best_move_after_wrong_move': opponent_best_move
            }
            exercises.append(exercise)
        else:
            # If the move was not considered as a "bad" move, just continue to the next move
            continue

    # Save the updated list of exercises
    with open(filename, 'w') as f:
        json.dump(exercises, f)
        
def create_and_save_all_exercises(username = "fanneett00669"):
    games = get_chess_com_games(username)
    print('All games loaded.')
    print('Creating exercises ...')
    for game in tqdm(games):
        analyze_game(game)

In [ ]:
create_and_save_all_exercises()

100%|██████████| 5/5 [00:01<00:00,  3.45it/s]


All games loaded.
Creating exercises ...


 49%|████▊     | 153/315 [14:09<15:13,  5.64s/it]

In [8]:
with open("exercises.json", 'r') as f:
    exercises = json.load(f)